# Topik: Analisis Sentimen Opini Publik terhadap Kebijakan Pemindahan Dana Rp 200 Triliun ke Bank Himbara oleh Menteri Keuangan Purbaya melalui Komentar YouTube

## Notebook: Preprocessing Data dari data hasil filtering.

**Anggota Kelompok:**  
- 140810230011 - Lukas Austin  
- 140810230045 - Devin Suryadi  
- 140810230057 - Orlando Bloem Sutono  

Mata Kuliah: Data Mining  
Kelas: A  
Dosen: Bu Helen

---

## 1. Import Library
Library yang dibutuhkan untuk preprocessing teks.

In [1]:
import os
import re
import time
import pandas as pd
import unicodedata
from bs4 import BeautifulSoup
import emoji
import json
from nltk.tokenize import word_tokenize
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from indoNLP.preprocessing import replace_word_elongation, replace_slang
# from indoNLP.preprocessing import remove_stopwords

print("Library berhasil dimuat!")

Library berhasil dimuat!


In [11]:
import nltk

print("Mendownload data NLTK...")
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
print("Download selesai!")

Mendownload data NLTK...
Download selesai!


## 2. Load Dataset
Memuat data hasil filtering komentar dari YouTube.

In [12]:
pd.set_option('display.max_colwidth', 200)

df = pd.read_csv("dataset_filtered.csv")
df = df[['comment']].copy()

print(f"Dataset dimuat: {len(df)} baris, kolom: {list(df.columns)}")
df.head()

Dataset dimuat: 1763 baris, kolom: ['comment']


,comment
0,"Seperti langit dan bumi, kerja pak Purbaya dan bu Sri mulyani. Senang lihat kerja Bpk. Sehat, semangat ya pak."
1,"Semoga Purbaya selamat, ngak dikerjain sama oligarki & para mafia di sektor energy, pangan dan keuangan yg merasa kepentingannya terganggu."
2,Ayooo pak pur ttp semangat bangun indonesia 👍
3,semoga usaha kecil dan daya beli meningkat. yaa
4,"Ah mulai besuk mo pinjem ke bank ah ,bye ito ito !!😅"


## 3. Text Cleaning
Melakukan pembersihan HTML, URL, emoji, dan normalisasi unicode.

In [13]:
# Menghapus HTML dan tag tidak relevan
def remove_html(text):
    return BeautifulSoup(str(text), "html.parser").get_text()

# Menormalisasi unicode ke bentuk konsisten
def normalize_unicode(text):
    return unicodedata.normalize('NFKC', str(text))

# Menghapus URL, mention, dan hashtag
def remove_url_mentions(text):
    return re.sub(r"http\S+|www\S+|@\w+|#\w+", " ", str(text))

# Menghapus emoji dari teks
def remove_emoji(text):
    return emoji.replace_emoji(str(text), replace=' ')
    
# Menormalkan spasi berlebih
def normalize_space(text):
    return re.sub(r'\s+', ' ', str(text)).strip()

In [14]:
df['clean_text'] = df['comment'].apply(remove_html)
df['clean_text'] = df['clean_text'].apply(normalize_unicode)
df['clean_text'] = df['clean_text'].apply(remove_url_mentions)
df['clean_text'] = df['clean_text'].apply(remove_emoji)
df['clean_text'] = df['clean_text'].apply(normalize_space)

df[['comment', 'clean_text']].head()

,comment,clean_text
0,"Seperti langit dan bumi, kerja pak Purbaya dan bu Sri mulyani. Senang lihat kerja Bpk. Sehat, semangat ya pak.","Seperti langit dan bumi, kerja pak Purbaya dan bu Sri mulyani. Senang lihat kerja Bpk. Sehat, semangat ya pak."
1,"Semoga Purbaya selamat, ngak dikerjain sama oligarki & para mafia di sektor energy, pangan dan keuangan yg merasa kepentingannya terganggu.","Semoga Purbaya selamat, ngak dikerjain sama oligarki & para mafia di sektor energy, pangan dan keuangan yg merasa kepentingannya terganggu."
2,Ayooo pak pur ttp semangat bangun indonesia 👍,Ayooo pak pur ttp semangat bangun indonesia
3,semoga usaha kecil dan daya beli meningkat. yaa,semoga usaha kecil dan daya beli meningkat. yaa
4,"Ah mulai besuk mo pinjem ke bank ah ,bye ito ito !!😅","Ah mulai besuk mo pinjem ke bank ah ,bye ito ito !!"


## 4. Case Folding
Mengubah semua teks menjadi huruf kecil (lowercase) untuk konsistensi.

In [15]:
# Case folding: Ubah ke lowercase
def case_folding(text):
    return str(text).lower()

df['clean_text'] = df['clean_text'].apply(case_folding)

df[['comment', 'clean_text']].head()

,comment,clean_text
0,"Seperti langit dan bumi, kerja pak Purbaya dan bu Sri mulyani. Senang lihat kerja Bpk. Sehat, semangat ya pak.","seperti langit dan bumi, kerja pak purbaya dan bu sri mulyani. senang lihat kerja bpk. sehat, semangat ya pak."
1,"Semoga Purbaya selamat, ngak dikerjain sama oligarki & para mafia di sektor energy, pangan dan keuangan yg merasa kepentingannya terganggu.","semoga purbaya selamat, ngak dikerjain sama oligarki & para mafia di sektor energy, pangan dan keuangan yg merasa kepentingannya terganggu."
2,Ayooo pak pur ttp semangat bangun indonesia 👍,ayooo pak pur ttp semangat bangun indonesia
3,semoga usaha kecil dan daya beli meningkat. yaa,semoga usaha kecil dan daya beli meningkat. yaa
4,"Ah mulai besuk mo pinjem ke bank ah ,bye ito ito !!😅","ah mulai besuk mo pinjem ke bank ah ,bye ito ito !!"


## 5. Tokenization
Memecah teks menjadi token (kata-kata individual).

In [16]:
# Tokenization menggunakan NLTK
def tokenize_text(text):
    return word_tokenize(str(text))

df['tokens'] = df['clean_text'].apply(tokenize_text)

# Tampilkan contoh hasil tokenization
print("Contoh hasil tokenization:")
print("=" * 80)
for i in range(3):
    print(f"\nTeks: {df.iloc[i]['clean_text']}")
    print(f"Tokens: {df.iloc[i]['tokens']}")
print("=" * 80)

df[['comment', 'clean_text', 'tokens']].head()

Contoh hasil tokenization:

Teks: seperti langit dan bumi, kerja pak purbaya dan bu sri mulyani. senang lihat kerja bpk. sehat, semangat ya pak.
Tokens: ['seperti', 'langit', 'dan', 'bumi', ',', 'kerja', 'pak', 'purbaya', 'dan', 'bu', 'sri', 'mulyani', '.', 'senang', 'lihat', 'kerja', 'bpk', '.', 'sehat', ',', 'semangat', 'ya', 'pak', '.']

Teks: semoga purbaya selamat, ngak dikerjain sama oligarki & para mafia di sektor energy, pangan dan keuangan yg merasa kepentingannya terganggu.
Tokens: ['semoga', 'purbaya', 'selamat', ',', 'ngak', 'dikerjain', 'sama', 'oligarki', '&', 'para', 'mafia', 'di', 'sektor', 'energy', ',', 'pangan', 'dan', 'keuangan', 'yg', 'merasa', 'kepentingannya', 'terganggu', '.']

Teks: ayooo pak pur ttp semangat bangun indonesia
Tokens: ['ayooo', 'pak', 'pur', 'ttp', 'semangat', 'bangun', 'indonesia']


,comment,clean_text,tokens
0,"Seperti langit dan bumi, kerja pak Purbaya dan bu Sri mulyani. Senang lihat kerja Bpk. Sehat, semangat ya pak.","seperti langit dan bumi, kerja pak purbaya dan bu sri mulyani. senang lihat kerja bpk. sehat, semangat ya pak.","[seperti, langit, dan, bumi, ,, kerja, pak, purbaya, dan, bu, sri, mulyani, ., senang, lihat, kerja, bpk, ., sehat, ,, semangat, ya, pak, .]"
1,"Semoga Purbaya selamat, ngak dikerjain sama oligarki & para mafia di sektor energy, pangan dan keuangan yg merasa kepentingannya terganggu.","semoga purbaya selamat, ngak dikerjain sama oligarki & para mafia di sektor energy, pangan dan keuangan yg merasa kepentingannya terganggu.","[semoga, purbaya, selamat, ,, ngak, dikerjain, sama, oligarki, &, para, mafia, di, sektor, energy, ,, pangan, dan, keuangan, yg, merasa, kepentingannya, terganggu, .]"
2,Ayooo pak pur ttp semangat bangun indonesia 👍,ayooo pak pur ttp semangat bangun indonesia,"[ayooo, pak, pur, ttp, semangat, bangun, indonesia]"
3,semoga usaha kecil dan daya beli meningkat. yaa,semoga usaha kecil dan daya beli meningkat. yaa,"[semoga, usaha, kecil, dan, daya, beli, meningkat, ., yaa]"
4,"Ah mulai besuk mo pinjem ke bank ah ,bye ito ito !!😅","ah mulai besuk mo pinjem ke bank ah ,bye ito ito !!","[ah, mulai, besuk, mo, pinjem, ke, bank, ah, ,, bye, ito, ito, !, !]"


## 6. Filtering
Menghapus tanda baca, angka, karakter berulang, dan kata 1 huruf dari setiap token.

In [17]:
# Filtering: hapus tanda baca, angka, kata 1 huruf, dan normalisasi elongasi
def filter_tokens(tokens):
    filtered = []
    for token in tokens:
        # Hapus tanda baca dan angka
        token = re.sub(r'[^a-z]', '', token)
        # Skip kata kosong atau 1 huruf
        if len(token) > 1:
            filtered.append(token)
    return filtered

# Normalisasi elongasi per token
def normalize_elongation_tokens(tokens):
    return [replace_word_elongation(token) for token in tokens]

df['tokens'] = df['tokens'].apply(filter_tokens)
df['tokens'] = df['tokens'].apply(normalize_elongation_tokens)

# Update clean_text dari tokens
df['clean_text'] = df['tokens'].apply(lambda x: ' '.join(x))

df[['comment', 'clean_text', 'tokens']].head()

,comment,clean_text,tokens
0,"Seperti langit dan bumi, kerja pak Purbaya dan bu Sri mulyani. Senang lihat kerja Bpk. Sehat, semangat ya pak.",seperti langit dan bumi kerja pak purbaya dan bu sri mulyani senang lihat kerja bpk sehat semangat ya pak,"[seperti, langit, dan, bumi, kerja, pak, purbaya, dan, bu, sri, mulyani, senang, lihat, kerja, bpk, sehat, semangat, ya, pak]"
1,"Semoga Purbaya selamat, ngak dikerjain sama oligarki & para mafia di sektor energy, pangan dan keuangan yg merasa kepentingannya terganggu.",semoga purbaya selamat ngak dikerjain sama oligarki para mafia di sektor energy pangan dan keuangan yg merasa kepentingannya terganggu,"[semoga, purbaya, selamat, ngak, dikerjain, sama, oligarki, para, mafia, di, sektor, energy, pangan, dan, keuangan, yg, merasa, kepentingannya, terganggu]"
2,Ayooo pak pur ttp semangat bangun indonesia 👍,ayo pak pur ttp semangat bangun indonesia,"[ayo, pak, pur, ttp, semangat, bangun, indonesia]"
3,semoga usaha kecil dan daya beli meningkat. yaa,semoga usaha kecil dan daya beli meningkat ya,"[semoga, usaha, kecil, dan, daya, beli, meningkat, ya]"
4,"Ah mulai besuk mo pinjem ke bank ah ,bye ito ito !!😅",ah mulai besuk mo pinjem ke bank ah bye ito ito,"[ah, mulai, besuk, mo, pinjem, ke, bank, ah, bye, ito, ito]"


## 7. Normalization
Mengganti kata tidak baku (slang) dan typo menjadi kata baku.

##### Daftar kata typo yang ada di dataset melalui eksplorasi dataset secara manual.

In [18]:
with open("typo_dict.json", "r", encoding="utf-8") as f:
    typo_dict = json.load(f)
    
test_data = dict(list(typo_dict.items())[:5])
test_data

{'bpk': 'bapak',
 'ngak': 'tidak',
 'sdh': 'sudah',
 'g': 'enggak',
 'ttp': 'tetap'}

##### Fungsi Normalisasi

In [19]:
# Normalisasi slang dan typo per token
def normalize_tokens(tokens):
    normalized = []
    for token in tokens:
        # Normalisasi slang
        token = replace_slang(token)
        # Normalisasi typo
        token = typo_dict.get(token, token)
        normalized.append(token)
    return normalized

In [20]:
df['tokens'] = df['tokens'].apply(normalize_tokens)

# Update clean_text dari tokens
df['clean_text'] = df['tokens'].apply(lambda x: ' '.join(x))

df[['comment', 'clean_text', 'tokens']].head()

,comment,clean_text,tokens
0,"Seperti langit dan bumi, kerja pak Purbaya dan bu Sri mulyani. Senang lihat kerja Bpk. Sehat, semangat ya pak.",seperti langit dan bumi kerja pak purbaya dan bu sri mulyani senang lihat kerja bapak sehat semangat nya pak,"[seperti, langit, dan, bumi, kerja, pak, purbaya, dan, bu, sri, mulyani, senang, lihat, kerja, bapak, sehat, semangat, nya, pak]"
1,"Semoga Purbaya selamat, ngak dikerjain sama oligarki & para mafia di sektor energy, pangan dan keuangan yg merasa kepentingannya terganggu.",semoga purbaya selamat tidak dikerjain saling oligarki para mafia disalurkan sektor energi pangan dan keuangan yang merasa kepentingannya terganggu,"[semoga, purbaya, selamat, tidak, dikerjain, saling, oligarki, para, mafia, disalurkan, sektor, energi, pangan, dan, keuangan, yang, merasa, kepentingannya, terganggu]"
2,Ayooo pak pur ttp semangat bangun indonesia 👍,ayo pak purbaya tetap semangat bangun indonesia,"[ayo, pak, purbaya, tetap, semangat, bangun, indonesia]"
3,semoga usaha kecil dan daya beli meningkat. yaa,semoga usaha kecil dan daya beli meningkat nya,"[semoga, usaha, kecil, dan, daya, beli, meningkat, nya]"
4,"Ah mulai besuk mo pinjem ke bank ah ,bye ito ito !!😅",ah mulai besok mau pinjam kepada bank ah bye ito ito,"[ah, mulai, besok, mau, pinjam, kepada, bank, ah, bye, ito, ito]"


## 8. Stopword Removal
Menghapus kata-kata umum yang tidak informatif (seperti "yang", "di", "ke", dll).

##### Inisialisasi Stopword dan Stemmer

In [21]:
# Inisialisasi Stopword
stop_factory = StopWordRemoverFactory()
stopwords = stop_factory.get_stop_words()

print(f"Total stopwords: {len(stopwords)}")
print(f"Contoh stopwords: {list(stopwords)[:20]}")

Total stopwords: 126
Contoh stopwords: ['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua', 'ia', 'seperti', 'jika', 'jika', 'sehingga', 'kembali', 'dan', 'tidak', 'ini', 'karena']


##### Proteksi kata negasi (agar tidak dihapus oleh stopword removal)

In [22]:
# Kata negasi yang perlu dilindungi
negation_words = ['tidak', 'bukan', 'belum', 'kurang', 'tanpa']

# Fungsi stopword removal dengan proteksi negasi
def remove_stopwords_tokens(tokens):
    return [token for token in tokens if token not in stopwords or token in negation_words]

##### Eksekusi stopword removal

In [23]:
df['tokens'] = df['tokens'].apply(remove_stopwords_tokens)

# Update clean_text dari tokens
df['clean_text'] = df['tokens'].apply(lambda x: ' '.join(x))

df[['comment', 'clean_text', 'tokens']].head()

,comment,clean_text,tokens
0,"Seperti langit dan bumi, kerja pak Purbaya dan bu Sri mulyani. Senang lihat kerja Bpk. Sehat, semangat ya pak.",langit bumi kerja pak purbaya bu sri mulyani senang lihat kerja bapak sehat semangat nya pak,"[langit, bumi, kerja, pak, purbaya, bu, sri, mulyani, senang, lihat, kerja, bapak, sehat, semangat, nya, pak]"
1,"Semoga Purbaya selamat, ngak dikerjain sama oligarki & para mafia di sektor energy, pangan dan keuangan yg merasa kepentingannya terganggu.",semoga purbaya selamat tidak dikerjain saling oligarki mafia disalurkan sektor energi pangan keuangan merasa kepentingannya terganggu,"[semoga, purbaya, selamat, tidak, dikerjain, saling, oligarki, mafia, disalurkan, sektor, energi, pangan, keuangan, merasa, kepentingannya, terganggu]"
2,Ayooo pak pur ttp semangat bangun indonesia 👍,ayo pak purbaya tetap semangat bangun indonesia,"[ayo, pak, purbaya, tetap, semangat, bangun, indonesia]"
3,semoga usaha kecil dan daya beli meningkat. yaa,semoga usaha kecil daya beli meningkat nya,"[semoga, usaha, kecil, daya, beli, meningkat, nya]"
4,"Ah mulai besuk mo pinjem ke bank ah ,bye ito ito !!😅",ah mulai besok mau pinjam bank ah bye ito ito,"[ah, mulai, besok, mau, pinjam, bank, ah, bye, ito, ito]"


## 9. Stemming
Mengubah kata ke bentuk dasarnya (misal: "berlari" -> "lari").

In [24]:
# Inisialisasi Stemmer
stem_factory = StemmerFactory()
stemmer = stem_factory.create_stemmer()

print("Stemmer berhasil diinisialisasi!")

Stemmer berhasil diinisialisasi!


In [25]:
# Stemming per token
def stem_tokens(tokens):
    return [stemmer.stem(token) for token in tokens]

df['tokens'] = df['tokens'].apply(stem_tokens)

# Update clean_text dari tokens
df['clean_text'] = df['tokens'].apply(lambda x: ' '.join(x))

df[['comment', 'clean_text', 'tokens']].head()

,comment,clean_text,tokens
0,"Seperti langit dan bumi, kerja pak Purbaya dan bu Sri mulyani. Senang lihat kerja Bpk. Sehat, semangat ya pak.",langit bumi kerja pak purbaya bu sri mulyani senang lihat kerja bapak sehat semangat nya pak,"[langit, bumi, kerja, pak, purbaya, bu, sri, mulyani, senang, lihat, kerja, bapak, sehat, semangat, nya, pak]"
1,"Semoga Purbaya selamat, ngak dikerjain sama oligarki & para mafia di sektor energy, pangan dan keuangan yg merasa kepentingannya terganggu.",moga purbaya selamat tidak dikerjain saling oligarki mafia salur sektor energi pangan uang rasa penting ganggu,"[moga, purbaya, selamat, tidak, dikerjain, saling, oligarki, mafia, salur, sektor, energi, pangan, uang, rasa, penting, ganggu]"
2,Ayooo pak pur ttp semangat bangun indonesia 👍,ayo pak purbaya tetap semangat bangun indonesia,"[ayo, pak, purbaya, tetap, semangat, bangun, indonesia]"
3,semoga usaha kecil dan daya beli meningkat. yaa,moga usaha kecil daya beli tingkat nya,"[moga, usaha, kecil, daya, beli, tingkat, nya]"
4,"Ah mulai besuk mo pinjem ke bank ah ,bye ito ito !!😅",ah mulai besok mau pinjam bank ah bye ito ito,"[ah, mulai, besok, mau, pinjam, bank, ah, bye, ito, ito]"


## 10. Menyimpan Dataset Cleaned
Menyimpan hasil preprocessing ke CSV dengan kolom sentiment kosong untuk labeling.

In [26]:
# Tambahkan kolom sentiment kosong
df['sentiment'] = ""

# Tampilkan hasil akhir preprocessing
print("PREPROCESSING SELESAI")
print(f"Total data: {len(df)} baris")
print(f"Kolom: {list(df.columns)}")
print("=" * 80)

df[['comment', 'clean_text', 'tokens', 'sentiment']].head()

PREPROCESSING SELESAI
Total data: 1763 baris
Kolom: ['comment', 'clean_text', 'tokens', 'sentiment']


,comment,clean_text,tokens,sentiment
0,"Seperti langit dan bumi, kerja pak Purbaya dan bu Sri mulyani. Senang lihat kerja Bpk. Sehat, semangat ya pak.",langit bumi kerja pak purbaya bu sri mulyani senang lihat kerja bapak sehat semangat nya pak,"[langit, bumi, kerja, pak, purbaya, bu, sri, mulyani, senang, lihat, kerja, bapak, sehat, semangat, nya, pak]",
1,"Semoga Purbaya selamat, ngak dikerjain sama oligarki & para mafia di sektor energy, pangan dan keuangan yg merasa kepentingannya terganggu.",moga purbaya selamat tidak dikerjain saling oligarki mafia salur sektor energi pangan uang rasa penting ganggu,"[moga, purbaya, selamat, tidak, dikerjain, saling, oligarki, mafia, salur, sektor, energi, pangan, uang, rasa, penting, ganggu]",
2,Ayooo pak pur ttp semangat bangun indonesia 👍,ayo pak purbaya tetap semangat bangun indonesia,"[ayo, pak, purbaya, tetap, semangat, bangun, indonesia]",
3,semoga usaha kecil dan daya beli meningkat. yaa,moga usaha kecil daya beli tingkat nya,"[moga, usaha, kecil, daya, beli, tingkat, nya]",
4,"Ah mulai besuk mo pinjem ke bank ah ,bye ito ito !!😅",ah mulai besok mau pinjam bank ah bye ito ito,"[ah, mulai, besok, mau, pinjam, bank, ah, bye, ito, ito]",


In [30]:
# Simpan ke CSV dengan kolom original_comment, comment (hasil preprocessing), dan sentiment
pd.DataFrame({
    'original_comment': df['comment'],  # Komentar asli dari dataset_filtered
    'comment': df['clean_text'],  # Hasil preprocessing (tokens yang sudah di-join)
    'sentiment': df['sentiment']
}).to_csv("dataset_cleaned.csv", index=False, encoding='utf-8')

print("Dataset berhasil disimpan dengan kolom: original_comment, comment, sentiment")

Dataset berhasil disimpan dengan kolom: original_comment, comment, sentiment


## 11. Labeling dengan API Gemini
Melakukan labeling otomatis pada seluruh data menggunakan model Gemini 2.0 Flash.
Prompt dibuat agar model memahami konteks kebijakan dan mengklasifikasikan opini publik secara akurat ke dalam tiga kelas sentimen: positif, negatif, dan netral.

##### API client dari Google Gemini.

In [16]:
from google import genai
from dotenv import load_dotenv

load_dotenv(override=True)
GEMINI_KEYS = [
    os.getenv("GEMINI_API_KEY_1"),
    os.getenv("GEMINI_API_KEY_2"),
    os.getenv("GEMINI_API_KEY_3"),
    os.getenv("GEMINI_API_KEY_4"),
    os.getenv("GEMINI_API_KEY_5")
]

# Index untuk tracking API key yang sedang digunakan
current_key_index = 0
client = genai.Client(api_key=GEMINI_KEYS[current_key_index])

print(f"Inisialisasi dengan API Key {current_key_index + 1}")
print(f"Total API keys tersedia: {len(GEMINI_KEYS)}")

Inisialisasi dengan API Key 1
Total API keys tersedia: 5


In [17]:
# Fungsi untuk switch ke API key berikutnya
def switch_api_key():
    global current_key_index, client
    current_key_index = (current_key_index + 1) % len(GEMINI_KEYS)
    client = genai.Client(api_key=GEMINI_KEYS[current_key_index])
    print(f" Beralih ke API Key {current_key_index + 1}")
    return client

##### Fungsi untuk prompt dan menjalankan pelabelan comment

In [18]:
def classify_sentiment_batch(comments, max_key_retries=3):
    global client
    
    # Tambahkan penomoran pada setiap komentar
    numbered_comments = "\n".join([f"{i+1}. {c}" for i, c in enumerate(comments)])

    # Prompt
    prompt = f"""
    Anda adalah analis sentimen publik. 
    Tugas Anda adalah menentukan apakah setiap komentar berikut memiliki sentimen positif, negatif, atau netral 
    terhadap kebijakan Menteri Keuangan Purbaya yang memindahkan dana Rp 200 triliun ke Bank Himbara. Pahami berita ini dengan baik sebelum menilai komentar, pahami juga sarkasme.
    Jumlah komentar ada {len(comments)}, pastikan jumlah akhirnya sama.

    Kriteria penilaian:
    - Positif berarti mendukung, memuji, atau menilai kebijakan ini baik/bermanfaat, mendukung dari sisi rakyat/pemerintah.
    - Negatif berarti menentang, mengkritik, atau menilai kebijakan ini buruk/merugikan, tidak mendukung kebijakan tersebut.
    - Netral berarti informatif / tidak informatif, bercanda, tidak menunjukkan opini jelas, tidak masuk ke yang dibahas.

    Berikut daftar komentar yang perlu diklasifikasikan:
    {numbered_comments}

    Kembalikan hasil dalam format Python list seperti berikut:
    ["Positif", "Negatif", "Netral", ...]
    Jumlah elemen dalam list harus sama dengan jumlah komentar ({len(comments)}).
    Jangan tambahkan teks lain, jangan tulis kode, jangan tulis ```python, jangan ada komentar tambahan.
    """

    # Tracking untuk rotasi key
    keys_tried = 0
    total_attempts = len(GEMINI_KEYS) * max_key_retries
    
    while keys_tried < total_attempts:
        try:
            response = client.models.generate_content(
                model="gemini-2.5-flash",
                contents=prompt
            )

            raw_output = response.text.strip()

            # Bersihkan tanda ``` jika gemini mengirimkan format yang tidak sesuai
            raw_output = re.sub(r"^```(?:python)?", "", raw_output)
            raw_output = re.sub(r"```$", "", raw_output).strip()

            # Parsing dengan ambil isi dalam tanda kutip
            labels = re.findall(r'"(.*?)"', raw_output)
            if not labels:  # Jika tidak ada tanda kutip, coba pisah berdasarkan koma
                labels = [w.strip(" []'\"\n") for w in raw_output.split(",") if w.strip()]
            
            # Validasi jumlah label
            if len(labels) != len(comments):
                print(f"Jumlah label ({len(labels)}) ≠ jumlah komentar ({len(comments)}) - Error parsing output")
                return None, False  # Return flag error parsing
            
            # Validasi isi label (harus salah satu dari sentimen yang valid)
            valid_sentiments = {"Positif", "Negatif", "Netral"}
            if not all(label in valid_sentiments for label in labels):
                invalid = [l for l in labels if l not in valid_sentiments]
                print(f"Label tidak valid ditemukan: {invalid[:3]}... - Error parsing output")
                return None, False  # Return flag error parsing
            
            return labels, True  # Berhasil
            
        except Exception as e:
            error_msg = str(e)
            keys_tried += 1
            
            # Cek apakah error karena limit/quota
            if "429" in error_msg or "quota" in error_msg.lower() or "resource_exhausted" in error_msg.lower() or "overloaded" in error_msg.lower():
                print(f"API Key {current_key_index + 1} kena limit. Mencoba key lain...")
                switch_api_key()
                time.sleep(2)  # Jeda sebentar sebelum coba key baru
            else:
                # Error lain, langsung return
                print(f"Error non-limit: {error_msg}")
                return None, False
    
    # Jika semua key sudah dicoba tapi masih gagal
    print(f"Semua API key gagal setelah {total_attempts} percobaan!")
    return None, False

##### Persiapan data untuk gemini dengan dataset yang sudah dibersihkan

In [19]:
# Baca dataset_cleaned.csv yang sudah jadi
df_original = pd.read_csv("dataset_cleaned.csv")

# Gunakan kolom 'original_comment' untuk labeling (komentar asli sebelum preprocessing)
print(f"Total komentar untuk labeling: {len(df_original)}")
print("\nContoh data:")
df_original.head()

Total komentar untuk labeling: 1763

Contoh data:


,original_comment,comment,sentiment
0,"Seperti langit dan bumi, kerja pak Purbaya dan...",langit bumi kerja pak purbaya bu sri mulyani s...,NaN
1,"Semoga Purbaya selamat, ngak dikerjain sama ol...",moga purbaya selamat tidak dikerjain saling ol...,NaN
2,Ayooo pak pur ttp semangat bangun indonesia 👍,ayo pak purbaya tetap semangat bangun indonesia,NaN
3,semoga usaha kecil dan daya beli meningkat. yaa,moga usaha kecil daya beli tingkat nya,NaN
4,"Ah mulai besuk mo pinjem ke bank ah ,bye ito i...",ah mulai besok mau pinjam bank ah bye ito ito,NaN


In [21]:
# Test dengan 5 komentar pertama dari original_comment
comments = df_original['original_comment'].head(10).tolist()
labels, success = classify_sentiment_batch(comments)

if success:
    df_preview = pd.DataFrame({
        "original_comment": comments,
        "sentiment": labels
    })
    print("Test berhasil!")
    display(df_preview)
else:
    print("Test gagal - periksa output Gemini atau coba lagi")

Test berhasil!


,original_comment,sentiment
0,"Seperti langit dan bumi, kerja pak Purbaya dan...",Positif
1,"Semoga Purbaya selamat, ngak dikerjain sama ol...",Positif
2,Ayooo pak pur ttp semangat bangun indonesia 👍,Positif
3,semoga usaha kecil dan daya beli meningkat. yaa,Positif
4,"Ah mulai besuk mo pinjem ke bank ah ,bye ito i...",Positif
5,pas bagian iklan bawaan channel gua mau next. ...,Netral
6,"Sudah lama usaha mandek, semoga dengan mentri ...",Positif
7,"Bang Bennix,.. bahas pandangan Prof. Latuhihin...",Netral
8,Bang Bennix menilai dari sisi fundamental peru...,Netral
9,yang jelaspenting banget kemampuan bayar masy...,Netral


In [22]:
def process_sentiment_labeling(df, column_name, batch_size=20, delay=5, max_batch_retries=5):
    all_labels = [] 
    
    # Loop batch
    for i in range(0, len(df), batch_size):
        # Ambil slice data
        batch_comments = df[column_name].iloc[i:i+batch_size].astype(str).tolist()
        batch_num = i//batch_size + 1
        total_batches = (len(df)-1)//batch_size + 1
        
        print(f"\n{'='*70}")
        print(f"Batch {batch_num}/{total_batches} ({len(batch_comments)} comments) | API Key: {current_key_index + 1}")
        
        # Retry mechanism untuk batch yang sama
        success = False
        
        for attempt in range(max_batch_retries):
            try:
                # Panggil fungsi klasifikasi
                labels, is_success = classify_sentiment_batch(batch_comments, max_key_retries=2)
                
                if is_success and labels:
                    # Berhasil
                    all_labels.extend(labels)
                    success = True
                    print(f"Batch berhasil diproses dengan sempurna")
                    break
                else:
                    # Error parsing atau output tidak valid
                    print(f"Percobaan {attempt + 1}/{max_batch_retries} gagal - Output Gemini tidak valid")
                    if attempt < max_batch_retries - 1:
                        wait_time = min(10 * (attempt + 1), 30)  # Incremental wait: 10s, 20s, 30s
                        print(f" Menunggu {wait_time} detik sebelum mencoba batch yang sama...")
                        time.sleep(wait_time)
                        continue
                
            except Exception as e:
                print(f"[Percobaan {attempt+1}/{max_batch_retries}] Error: {e}")
                if attempt < max_batch_retries - 1:
                    wait_time = min(10 * (attempt + 1), 30)
                    print(f" Menunggu {wait_time} detik sebelum mencoba batch yang sama...")
                    time.sleep(wait_time)

        # Jika setelah max_batch_retries percobaan tetap gagal
        if not success:
            print(f"[FATAL] Batch {batch_num} gagal total setelah {max_batch_retries} percobaan. Mengisi dengan 'Error'.")
            all_labels.extend(["Error"] * len(batch_comments))
        
        # Jeda antar batch
        if i + batch_size < len(df):
            print(f" Jeda {delay} detik sebelum batch berikutnya...")
            time.sleep(delay)

    print(f"\n{'='*70}")
    print(f"Labeling selesai! Total komentar diproses: {len(all_labels)}")
    
    # Gabungkan ke DataFrame
    df_result = df.copy()
    df_result['sentiment'] = all_labels
    return df_result

In [23]:
# Jalankan labeling untuk semua data menggunakan original_comment
df_result = process_sentiment_labeling(df_original, 'original_comment', batch_size=50, delay=120)


Batch 1/36 (50 comments) | API Key: 2
API Key 2 kena limit. Mencoba key lain...
API Key 2 kena limit. Mencoba key lain...
 Beralih ke API Key 3
 Beralih ke API Key 3
API Key 3 kena limit. Mencoba key lain...
API Key 3 kena limit. Mencoba key lain...
 Beralih ke API Key 4
 Beralih ke API Key 4
Batch berhasil diproses dengan sempurna
 Jeda 120 detik sebelum batch berikutnya...
Batch berhasil diproses dengan sempurna
 Jeda 120 detik sebelum batch berikutnya...

Batch 2/36 (50 comments) | API Key: 4

Batch 2/36 (50 comments) | API Key: 4
API Key 4 kena limit. Mencoba key lain...
API Key 4 kena limit. Mencoba key lain...
 Beralih ke API Key 5
 Beralih ke API Key 5
Batch berhasil diproses dengan sempurna
 Jeda 120 detik sebelum batch berikutnya...
Batch berhasil diproses dengan sempurna
 Jeda 120 detik sebelum batch berikutnya...

Batch 3/36 (50 comments) | API Key: 5

Batch 3/36 (50 comments) | API Key: 5
Batch berhasil diproses dengan sempurna
 Jeda 120 detik sebelum batch berikutnya...
B

In [ ]:
# Retry data error
# Ambil baris yang gagal
error_mask = df_result['sentiment'] == "Error"
df_error = df_result[error_mask].copy()

print(f"Jumlah komentar error yang akan diulang: {len(df_error)}")

if len(df_error) > 0:
    # Jalankan ulang labeling hanya untuk data error menggunakan original_commentz``
    retry_result = process_sentiment_labeling(
        df=df_error,
        column_name="original_comment",
        batch_size=25, 
        delay=60
    )
    # Gantikan label Error dengan hasil retry
    df_result.loc[error_mask, 'sentiment'] = retry_result['sentiment'].values
    print("Retry selesai. Label Error telah diperbarui.")
else:
    print("Tidak ada data Error. Tidak perlu retry.")

Jumlah komentar error yang akan diulang: 100

Memproses batch 1 (25 komentar)...
Menunggu 60 detik sebelum lanjut ke batch berikutnya.
Menunggu 60 detik sebelum lanjut ke batch berikutnya.

Memproses batch 2 (25 komentar)...

Memproses batch 2 (25 komentar)...
Menunggu 60 detik sebelum lanjut ke batch berikutnya.
Menunggu 60 detik sebelum lanjut ke batch berikutnya.

Memproses batch 3 (25 komentar)...

Memproses batch 3 (25 komentar)...
Menunggu 60 detik sebelum lanjut ke batch berikutnya.
Menunggu 60 detik sebelum lanjut ke batch berikutnya.

Memproses batch 4 (25 komentar)...

Memproses batch 4 (25 komentar)...
Retry selesai. Label Error telah diperbarui.
Retry selesai. Label Error telah diperbarui.


In [24]:
# Simpan hasil labeling ke dataset_labeled.csv
df_result.to_csv("dataset_labeled.csv", index=False, encoding="utf-8")

print(f"Dataset final berhasil disimpan ke dataset_labeled.csv")
print(f"Total baris: {len(df_result)}")
print(f"\nDistribusi sentimen:")
print(df_result['sentiment'].value_counts())

Dataset final berhasil disimpan ke dataset_labeled.csv
Total baris: 1763

Distribusi sentimen:
sentiment
Negatif    720
Positif    684
Netral     359
Name: count, dtype: int64
